# 8. Interpolation

*If you haven't already installed the packages `numpy`, `pandas`, `xlrd` and `scipy`, please do so.*

This session has no new material per se. It's supposed to combine elements from the previous sessions into a larger exercise.  

The exercise is about 3D interpolation. A set of known points $(x_{known}, x_{known}, z_{known})$ have prescribed values and are used as basis for interpolating $z$-values for a large set of points where only $(x, y)$ are known.

For performing the actual interpolation, we call a function from a third party library called `scipy`. It's built on top of of `numpy` and holds many operations used in scientific analysis. 

The code originates from a COWI project where the points $(x, y)$ represent node coordinates from a base slab in a Finite Element model, while $z$-coordinates denote settlement values. The known points $(x_{known}, y_{known}, z_{known})$ stem from a detailed geotechnical analysis which could only be performed in a certain amount of points. The settlement values in the remaining points $(x, y)$ were therefore put into the FE-model as imposed displacements by a procedure similar to this. 


# Exercise 1.1
Read through the code given in the script below and try to understand what it does and how it does it.

Copy the script to your editor and run it. 

Add print statements if you are unsure about how a certain variable looks at any point throughout the code. Remember you can print the first five rows of a DataFrame with `df.head()`.    

The script reads two Excel files, i.e. one containing known points and one containing points to be interpolated. 
One limitation of Excel files is that they cannot be read while they are open. If you want to inspect these files while running the script, create a copy. 

# Exercise 1.2
The bulk of the computational work in the script is done by the line:

---
```python
settlements_interpolated = griddata(xy_known, settlements_known, (x_nodes, y_nodes), method='cubic')
```
---
This is the `scipy` function that performs the interpolation. Try to read the documentation for it [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.griddata.html)

The returned value that we save in the variable `settlements_interpolated` is a numpy array. You can see this by `print(type(settlements_interpolated))` which returns: `<class 'numpy.ndarray'>`. 

The last part of the given code creates a figure object and an axis object which enables 3D plots.

**Continue the plotting code to add:**

* 3D scatter plot of the known points $(x_{known}, y_{known}, z_{known})$

* 3D scatter plot of the interpolated points $(x, y, z)$

The plots should all be in the same figure and axis. Adding a scatter plot to an axis `ax` can be done as `ax.scatter(...)`.

# Exercise 1.3

As mentioned, this was used in a project for interpolating settlement values to be applied in an FE-model. The FE-software (Sofistik) has a certain input language which the interpolated values needed to be blended into. 

In this exercise we will construct the input `.dat`.file that Sofistik can read. 

>**Note:** This procedure could be used to produce many file types. It's a good way to programmatically create input files to software. This is just one specific example of a use case.

The Sofistik input file we want to create has this syntax:

---

<pre><code>
+PROG SOFILOAD 

LC 25 type 'SL' fact 1.0 facd 0.0 titl 'LT settlement all nodes'

  POIN NODE <font color='#1E90FF'>insert first node number</font> WIDE 0 TYPE WZZ <font color='#1E90FF'>insert first interpolated z-value</font>
  ...
  <font color='#1E90FF'><i>one line per pair of node number/z-value</i></font> 
  ...
  POIN NODE <font color='#1E90FF'>insert last node number</font> WIDE 0 TYPE WZZ <font color='#1E90FF'>insert last interpolated z-value</font>
  
END
</code></pre>

---

The indented block should print all the node/settlement pairs. The three non-indented lines should only appear once. The output file should look like the file `interpolation_output_example.dat` in the folder. Newlines are made by `\n`.


## How to write to files
To write data to a file we can use something called a **context manager**. Basically, it allows us to open a file and write to it. See code snippet below:

---
~~~python
# Use a context manager to open and write ('w') to file
with open('file_name.dat', 'w') as file:
    
    # The file can from here on out be referred to as file
    file.write("This text will be written inside 'file_name.dat'")
~~~
---

By using the concept the file is automatically closed after our indented block is terminated. It also creates the file in case it doesn't already exist. 


# The script

~~~python

import pandas as pd
import numpy as np
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


# Set name of Excel file to read containing known points
file_known = 'known_points.xlsx'

# Set name of sheet to read from Excel file
sheet_known = 'Sheet1'

# Read data from Excel sheet into a dataframe
df = pd.read_excel(file_known, sheet_name=sheet_known, skiprows=7)

# Extract column names starting with 'Y' into new dataframe of known Y-coords
df_y = df[df.columns[df.columns.str.startswith('Y')]]

# Extract column names starting with 'Z' into new dataframe of known Z-coords
df_z_known = df[df.columns[df.columns.str.startswith('Z')]]

# Flatten dataframe values into 1D array (matri format -> vector format)
y_known = df_y.values.flatten()
z_known = df_z_known.values.flatten()

# Extract known x-values
x_known = df['X']

# Create X-array by repeating itself as many times as there are Y-columns
# This will create matching(x, y)-points between arrays x and y
x_known = np.repeat(x_known, len(df_y.columns))

# Mirror known y-values and add corresponding x- and y-values
x_known = np.append(x_known, x_known)
y_known = np.append(y_known, -y_known)
z_known = np.append(z_known, z_known)

# Arrange known (x, y) points to fit input for interpolation
xy_known = np.array(list(zip(x_known, y_known)))

# Set names and read Excel file with nodes to be interpolated
file_nodes = 'points_to_be_interpolated.xlsx'
sheet_nodes = 'XLSX-Export'
df_nodes = pd.read_excel(file_nodes, sheet_name=sheet_nodes)

# Extract x- and y-coordinates of nodes to be interpolated
x_nodes = df_nodes['X [m]']
y_nodes = df_nodes['Y [m]']

# Extract node numbers for points to be interpolated
node_no = df_nodes['NR']

# Perform interpolation calculation
points_interpolated = griddata(xy_known, z_known, (x_nodes, y_nodes), method='cubic')


####################
### Exercise 1.2 ###
####################
# Create figure object
fig = plt.figure()

# Create axis object for 3D plot
ax = fig.add_subplot(111, projection='3d')

# Plot known points as 3D scatter plot (ax.scatter(...))
    # <Put plotting code here!>

# Plot interpolated points as 3D scatter plot
    # <Put plotting code here!>

# Show figure
    # <Put plotting code here!>


####################
### Exercise 1.3 ###
####################
# Write Sofistik input code to .dat-file for applying the interpolated z-values as 
# imposed displacement load in all points (x, y)
    # <Put code that creates and writes to a .dat file here!>  

~~~

# End of exercises

*The cell below is for setting the style of this document. It's not part of the exercises.*

In [1]:
from IPython.display import HTML
HTML('<style>{}</style>'.format(open('../css/cowi.css').read()))